<a href="https://colab.research.google.com/github/wigglytuff-tu/Analytics-Coords/blob/main/TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Transfer Learning**
In this Notebook we will go through the process of Transfer learning, writing the code from scratch for an Image Classification Task. We will we be using the following dataset: https://www.kaggle.com/itsahmad/indoor-scenes-cvpr-2019/code. Since most categories have only 100 images which typically isn’t enough for a neural network to learn to high accuracy. Therefore, instead of building and training a CNN from scratch, we’ll use a pre-built and pre-trained model applying transfer learning. we use a model which has been trained on a larger dataset and use the pre-trained to predict over our smaller dataset.The idea is the convolutional layers extract general, low-level features that are applicable across images — such as edges, patterns, gradients — and the later layers identify specific features within an image such as eyes or wheels.

In [1]:
#Import the Required Libraries
import pandas as pd
import numpy as np
import os

import torch
import torchvision
from torch.optim import Adam
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms

In [ ]:
#Getting the Dataset
from google.colab import drive
drive.mount('/content/gdrive')

In [3]:
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle" #Directing the environment to kaggle Folder
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [5]:
!kaggle datasets download -d itsahmad/indoor-scenes-cvpr-2019

100% 2.33G/2.34G [00:36<00:00, 70.8MB/s]
100% 2.34G/2.34G [00:36<00:00, 69.2MB/s]


In [6]:
!ls
!unzip \*.zip  && rm *.zip

In [4]:
data_dir  = './indoorCVPR_09/Images'
classes = os.listdir(data_dir)
print(classes)
len(classes)


['airport_inside', 'artstudio', 'auditorium', 'bakery', 'bar', 'bathroom', 'bedroom', 'bookstore', 'bowling', 'buffet', 'casino', 'children_room', 'church_inside', 'classroom', 'cloister', 'closet', 'clothingstore', 'computerroom', 'concert_hall', 'corridor', 'deli', 'dentaloffice', 'dining_room', 'elevator', 'fastfood_restaurant', 'florist', 'gameroom', 'garage', 'greenhouse', 'grocerystore', 'gym', 'hairsalon', 'hospitalroom', 'inside_bus', 'inside_subway', 'jewelleryshop', 'kindergarden', 'kitchen', 'laboratorywet', 'laundromat', 'library', 'livingroom', 'lobby', 'locker_room', 'mall', 'meeting_room', 'movietheater', 'museum', 'nursery', 'office', 'operating_room', 'pantry', 'poolinside', 'prisoncell', 'restaurant', 'restaurant_kitchen', 'shoeshop', 'stairscase', 'studiomusic', 'subway', 'toystore', 'trainstation', 'tv_studio', 'videostore', 'waitingroom', 'warehouse', 'winecellar']


67

We are done with the first step, i.e. importing our Dataset. Now we will preprocess our dataset to use with our pre-trained ImageNet Model.Each model will have different input requirements, but if we read through what Imagenet requires, we figure out that our images need to be 224x224 and normalized to a range.To process an image in PyTorch, we use ```transforms``` , simple operations applied to arrays. The validation (and testing) transforms are as follows:


*   Resize

*   Center crop to 224 x 224
*   Convert to a tensor


*   Normalize with mean and standard deviation



In [5]:
# Image transformations
image_transforms = transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),  # Image net standards
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards
    ])


In [6]:
#Creating Datasets and Dataloaders
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

# Datasets from folders
data = datasets.ImageFolder(data_dir, transform=image_transforms)    

train_ds, val_ds, test_ds = torch.utils.data.random_split(data, [13500, 1500, 620])
len(train_ds), len(val_ds), len(test_ds)

# Dataloader iterators, make sure to shuffle
dataloaders = {
    'train': DataLoader(train_ds, batch_size=32, shuffle=True),
    'val': DataLoader(val_ds, batch_size=32, shuffle=True)
}

In [ ]:
# Iterate through the dataloader once
'''trainiter = iter(dataloaders['train'])
features, labels = next(trainiter)
features.shape, labels.shape'''

# **Pre-Trained Models for Image Recognition**
With our data in shape, we next turn our attention to the model. For this, we’ll use a pre-trained convolutional neural network. PyTorch has a number of models that have already been trained on millions of images from 1000 classes in [Imagenet](http://www.image-net.org/). The complete list of models can be [seen here](https://pytorch.org/docs/stable/torchvision/models.html). The performance of these models on Imagenet is shown below:

![image.png](https://miro.medium.com/max/942/1*0W310-cMNHPWjErqPuGXpw.png)

Here, we will use Inception v3. Now we will follow these steps:

1.   Load in pre-trained weights from a network trained on a large dataset
2.   Freeze all the weights in the lower (convolutional) layers: the layers to freeze are adjusted depending on similarity of new task to original dataset

1.   Replace the upper layers of the network with a custom classifier: the number of outputs must be set equal to the number of classes (here 67 classes)
2.   Train only the custom classifier layers for the task thereby optimizing the model for smaller dataset







In [11]:
#Loading in pre-trained Model
from torchvision import models
model = models.inception_v3(pretrained = True)

#You can use ```dir(models)``` to see various models available for Transfer Learning

In [8]:
# Freeze model weights
for param in model.parameters():
    param.requires_grad = False
#print(model)

In [9]:
#Add the following layer with 67 output fetures as we have 67 classes


num_ftrs = model.AuxLogits.fc.in_features
model.AuxLogits.fc = nn.Linear(num_ftrs, 67)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 67)
model = model.to('cuda')  #Moving Model to GPU